## Consulta

In [ ]:
import csv
import pathlib
import consultas_module

CUSTOM_AIRPORTS = pathlib.Path("../datasets/custom_datasets/ar-airports.csv")
ARGENTINA_CITIES = pathlib.Path("../datasets/original_datasets/ar.csv")
CUSTOM_CONECTIVIDAD = pathlib.Path(
    "../datasets/custom_datasets/Conectividad_Internet.csv"
)
CUSTOM_POBLACION = pathlib.Path(
    "../datasets/custom_datasets/c2022_tp_c_resumen_adaptado.csv"
)

In [ ]:
import pandas as pd

df_airports = pd.read_csv(CUSTOM_AIRPORTS)

**1**. Los diferentes tipos de aeropuertos según la columna 'type'

In [ ]:
# Open database and creates a list that will used multiple times.
with CUSTOM_AIRPORTS.open(mode="r", newline="", encoding="utf-8") as read_airports:
    airports = csv.reader(read_airports)
    next(airports)
    list_airports = list(airports)

# Database design
AIRPORT_TYPE = 2
AIRPORT_NAME = 3
ELEVATION_FT = 6
ELEVATION_TYPE = -2

In [ ]:
# Prints the set of airports in airports database
print("Tipos de aeropuertos:")
for AIRPORT_TYPE in set(map(lambda x: x[AIRPORT_TYPE], list_airports)):
    print(f"\u2022 {AIRPORT_TYPE}")

In [ ]:
print("Tipos de aeropuertos:")
for air_type in df_airports.type.unique():
    print(f"\u2022 {air_type}")

**2**. Los aeropuertos de una elevación, (bajo, medio, alto) cuya especificación se permita modificar fácilmente, evaluando la columna creada 'elevation_name'.

In [ ]:
while True:
    # Requires elevation type search input
    elevation = input(
        f"Indique la elevación del aeropuerto buscada: \n 1️⃣ bajo,\n 2️⃣ medio o\n 3️⃣ altos"
    )
    # Verifies that the input is valid
    if elevation == "1" or elevation == "2" or elevation == "3":
        match int(elevation):
            case 1:
                elevation = "bajo"
            case 2:
                elevation = "medio"
            case 3:
                elevation = "altos"
        break
    print("Por favor, ingrese 1️⃣, 2️⃣ o 3️⃣")
print(f"Los siguientes aeropuertos se encuentran a una elevación {elevation}:")
for line in filter(lambda airport: airport[ELEVATION_TYPE] == elevation, list_airports):
    print(f"\u2022 {line[AIRPORT_NAME]}")

In [ ]:
df_airports[df_airports.elevation_name == elevation]

**3**. Los aeropuertos que tienen una mayor o menor elevación con respecto al valor numérico dado.

In [ ]:
while True:
    # Requires greater or smaller input
    comparison = input("¿Busca un valor 1️⃣ mayor o 2️⃣ menor?")
    # Verifies that the input is valid
    if comparison == "1" or comparison == "2":
        match int(comparison):
            case 1:
                comparison = "mayor"
            case 2:
                comparison = "menor"
        break
    print("Ingrese 1️⃣ mayor o 2️⃣ menor")
while True:
    # Requires elevation input
    elev_mt = input("Indique la elevación (en metros) a comparar")
    # Verifies that the input is valid
    if elev_mt.isdigit():
        elev_mt = int(elev_mt)
        elev_ft = elev_mt / 0.3048
        break
    print("Ingrese un número")
print(f"Los aeropuertos con una altura {comparison} a {elev_mt} metros son:")
for line in list_airports:
    # Verifies that the dataset includes the elevation ft
    if line[ELEVATION_FT]:
        if (
            comparison == "mayor"
            and int(line[ELEVATION_FT]) > elev_ft
            or comparison == "menor"
            and int(line[ELEVATION_FT]) < elev_ft
        ):
            # Prints matches
            print(f"\u2022 {line[AIRPORT_NAME]}")

In [ ]:
df_elevation_airports = (
    df_airports[df_airports.elevation_ft > elev_ft]
    if comparison == "mayor"
    else df_airports[df_airports.elevation_ft < elev_ft]
)
df_elevation_airports

**4**. Los aeropuertos, lagos y tipo de conectividad en provincias con población mayor o menor a un valor que se pueda especificar fácilmente 

In [ ]:
option = input(
    f"""Ingresar número de poblacion junto a:
				+"num" para mostrar poblacion mayor al valor
				-"num" para mostrar poblacion menor al valor"""
)
# Verifico que el input sea correcto, UN solo signo seguido de numeros.
if option[0] == "+" or option[0] == "-":
    num = option[1:]
    if num.isdigit():
        # Identifico provincias que cumplen el requisito
        province = (
            consultas_module.filter_provinces(num, "mayor")
            if option[0] == "+"
            else consultas_module.filter_provinces(num, "menor")
        )
        # Imprimo aeropuertos, lagos y conectividad de las provincias que cumplen
        consultas_module.show_info(province)
    else:
        print(
            "Formato input incorrecto, debe ser UN solo [+ o -] seguido por un numero."
        )
else:
    print(
        "Input incorrecto, debe comenzar especificando si es +(mayor) o -(menor) seguido por un numero."
    )

**5**. Mostrar los aeropuertos en las capitales de cada provincia

_-Recopilo el nombre de la capital de cada provincia._

In [ ]:
listaCiudades = []
with ARGENTINA_CITIES.open(mode="r", newline="", encoding="utf-8") as read_a:
    a = csv.reader(read_a)
    next(a)
    for i in a:
        if i[6] == "admin":
            # Ciudades capital
            listaCiudades.append(i[0])

_-Imprimo la información de los aeropuertos que se encuentran en la capital de cada provincia._

In [ ]:
with CUSTOM_AIRPORTS.open(mode="r", newline="", encoding="utf-8") as read_airports:
    airports = csv.DictReader(read_airports)
    header = next(airports)
    for i in airports:
        if i["municipality"] in listaCiudades:
            # Imprimo solo la informacion mas importante
            print(
                f"""\n
	id               : {i["id"]}
	type             : {i["type"]}
	name             : {i["name"]}
	country_name     : {i["country_name"]}
	provincia        : {i["region_name"][:i["region_name"].rfind("Province")]}
	ciudad           : {i["municipality"]}"""
            )

**6**. Mostrar los Lagos de una superficie según la columna 'Sup Tamaño' donde el criterio (chico, medio, grande) se puede indicar fácilmente

_-A partir del input del usuario muestro la información sobre los lagos correspondientes_

In [ ]:
menu = input(
    f"""
		----Elegir una opción:----
			1. Chico
			2. Medio
			3. Grande 
			Ingresa tu opcion : """
)
if not menu.isdigit():
    print("\n** Opción Inválida.\n")
else:
    if int(menu) == 1:
        consultas_module.show_lake_info("chico")
    elif int(menu) == 2:
        consultas_module.show_lake_info("medio")
    elif int(menu) == 3:
        consultas_module.show_lake_info("grande")
    else:
        print("\n** Opción inválida.\n")

**7**. Listar las 5 Jurisdicciones con mayor porcentaje de población en situación de calle.


In [ ]:
jurisdiccion = 0
varones_poblacion = 5
mujeres_poblacion = 9
porcentaje_en_situcacion_calle = 13

In [ ]:
with CUSTOM_POBLACION.open(mode="r", newline="", encoding="utf-8") as read_poblacion:
    population = csv.reader(read_poblacion)
    # Salto el header y la linea del total pais
    next(population)
    next(population)
    list_population = list(population)
    # Ordeno la lista de mayor a menor segun la poblacion en situacion de calle
    ordered_population = sorted(
        list_population, key=lambda x: x[porcentaje_en_situcacion_calle], reverse=True
    )
    print(
        "Las 5 Jurisdicciones con mayor porcentaje de población en situación de calle son:"
    )
    # Recorro las cinco primeras posiciones en la lista ordenada
    for i in range(5):
        print(
            f"-Jurisdiccion: {ordered_population[i][jurisdiccion]}, porcentaje de poblacion en situacion de calle: {ordered_population[i][porcentaje_en_situcacion_calle]}"
        )

**8**. Obtener la jurisdicción donde la brecha entre las poblaciones diferenciadas según sexo registrado al nacer es mayor.

In [ ]:
juris = ""
max = 0
with CUSTOM_POBLACION.open(mode="r", newline="", encoding="utf-8") as read_poblacion:
    population = csv.reader(read_poblacion)
    # Salto el header y la linea del total pais
    next(population)
    next(population)
    for line in population:
        # Obtengo la diferencia entre poblacion varones y poblacion de mujeres
        gap = consultas_module.calculate_gap(
            line[varones_poblacion], line[mujeres_poblacion]
        )
        if max < gap:
            juris = line[jurisdiccion]
            max = gap
    print(
        f"{juris} tiene la mayor brecha entre las poblaciones diferenciadas según sexo registrado al nacer con {max} de brecha"
    )

**9**. Mostrar los diferentes tipos de conectividades.

*Abro el archivo y con el header obtengo los diferentes tipos de conectividades*


In [ ]:
with CUSTOM_CONECTIVIDAD.open(mode="r", newline="", encoding="utf-8") as reader_conect:
    conect = csv.reader(reader_conect)
    # Obetengo el header
    header = next(conect)
    print("Los diferentes tipos de conectividades son: ")
    # Itero en la lista desde la posicion 4 hasta la 13(se procesa hasta la 12)
    for conectivity in header[4:13]:
        print(f"-{conectivity}")

**10**. Mostrar las cantidades de localidades con cada tipo de conectividad

In [ ]:
with CUSTOM_CONECTIVIDAD.open(
    mode="r", newline="", encoding="utf-8"
) as read_connections:
    connectivity_map = csv.DictReader(read_connections, delimiter=",")
    connections_header = connectivity_map.fieldnames
    connectivity_types = connections_header[4:13:]

_Se inicia un diccionario que va a servir como contador para resolver el enunciado_

In [ ]:
connections_quantity = {}

for connection in connectivity_types:
    connections_quantity[connection] = 0

_Se abre el archivo y se van contando por cada ciudad si tiene cada tipo de conexion para despues devolver un total_

In [ ]:
with CUSTOM_CONECTIVIDAD.open(
    mode="r", newline="", encoding="utf-8"
) as read_connections:

    connectivity_map = csv.DictReader(read_connections, delimiter=",")

    for city in connectivity_map:
        for connection in connectivity_types:
            if city[connection] == "SI":
                connections_quantity[connection] += 1

    print(connections_quantity)

**11**. Mostrar las provincias para las cuales todas sus ciudades poseen FIBRA ÓPTICA.

_Se abre el archivo y se pasa su contenido a una lista la cual vamos a iterar para resolver la consulta_

In [ ]:
with CUSTOM_CONECTIVIDAD.open(
    mode="r", newline="", encoding="utf-8"
) as read_connections:

    connectivity_map = csv.reader(read_connections, delimiter=",")

    next(connectivity_map)

    connections = list(connectivity_map)

_\Se inicializan las estructuras con las cuales vamos a trabajar_\

In [ ]:
province_already_read = []
province_not_all_of = []
connections.sort(key=lambda x: x[0])

provinces = list(set(map(lambda connect: connect[0], connections)))

provinces.sort()

province_iter = iter(provinces)

connections = iter(connections)

province = next(province_iter)

_Se recorren todas las conexiones y las provincias para buscar que provincias no tienen fibra optica en la totalidad de su provincia_

In [ ]:
for connect in connections:
    current_province = province
    if current_province == province and connect[0] not in province_already_read:
        if (
            current_province == province
            and connect[0] not in province_already_read
            and connect[7] == "NO"
        ):
            province_already_read.append(connect[0])
            province_not_all_of.append(connect[0])
            province = next(province_iter)
        else:
            continue

_Se crea una lista con las provincias que SI tienen fibra optica en su totalidad_

In [ ]:
province_with_optical_fiber = list(
    filter(lambda province: province not in province_not_all_of, provinces)
)

province_with_optical_fiber

**12**. Mostrar para cada provincia su capital y, si se conoce la información para dicha
capital, informar si posee conectividad (campo 'posee_conectividad' creado
previamente). En caso de no conocer la información mostrar el texto “conectividad
desconocida”.

_Se abre el archivo de las ciudades argentinas para tener una estructura de datos que contenga la capital de cada provincia y se parsea el nombre de la provincia "TIERRA DEL FUEGO" para que coincida con el archivo Conectividad_Internet.csv_

In [ ]:
import consultas_module as consulta


with ARGENTINA_CITIES.open(mode="r", newline="", encoding="utf-8") as read_cities:

    cities = csv.DictReader(read_cities, delimiter=",")

    provinces_and_capital = {}

    for city in cities:
        if city["capital"] == "admin":
            city["admin_name"] = consulta.replace_accented_letters(
                city["admin_name"]
            ).upper()
            if (
                city["admin_name"]
                == "TIERRA DEL FUEGO, ANTARTIDA E ISLAS DEL ATLANTICO SUR"
            ):
                city["admin_name"] = "TIERRA DEL FUEGO"
            provinces_and_capital[city["admin_name"]] = city["city"]

_Se recorren las ciudades buscando la capital de cada provincia para resolver la consulta_

In [ ]:
informed_provinces = []
with CUSTOM_CONECTIVIDAD.open(
    mode="r", newline="", encoding="utf-8"
) as read_connections:
    connectivity_map = csv.DictReader(read_connections, delimiter=",")

    for city_connectivity in connectivity_map:
        if (
            city_connectivity["Localidad"] in provinces_and_capital.values()
            and city_connectivity["Localidad"]
            == provinces_and_capital[city_connectivity["Provincia"]]
        ):
            if city_connectivity["posee_conectividad"] == "SI":
                print(
                    f"La ciudad {city_connectivity['Localidad']}, capital de {city_connectivity['Provincia']}, posee conectividad: {city_connectivity['posee_conectividad']}."
                )
                informed_provinces.append(city_connectivity["Provincia"])

for province in provinces_and_capital.keys():
    if province not in informed_provinces:
        print(f"Conectividad desconocida en la capital de {province}")